<a href="https://colab.research.google.com/github/Samoed/CourseWork2/blob/main/Test_lenngth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -U bert-extractive-summarizer
!pip3 install -U transformers
!pip3 install -U pdfminer.six

     |████████████████████████████████| 2.0MB 24.7MB/s 
     |████████████████████████████████| 3.2MB 43.3MB/s 
     |████████████████████████████████| 890kB 43.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=cd89f3d787de44680b845f17f67a161e4fa600afebacef8b3af0e12bc77a00db
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.4.2)
     |████████████████████████████████| 5.6MB 16.5MB/s 
     |████████████████████████████████| 3.2MB 43.6MB/s 


In [2]:
!unzip embeddingsShort.zip

Archive:  embeddingsShort.zip
  inflating: embeddingsPickle/ Абашева Александра Сергеевна.data  
  inflating: embeddingsPickle/ Андреева Ольга Юрьевна.data  
  inflating: embeddingsPickle/ Артемьев Дмитрий Геннадьевич.data  
  inflating: embeddingsPickle/ Барахас Анхель.data  
  inflating: embeddingsPickle/ Белых Светлана Александровна.data  
  inflating: embeddingsPickle/ Божья-Воля Анастасия Александровна.data  
  inflating: embeddingsPickle/ Бузмаков Алексей Владимирович.data  
  inflating: embeddingsPickle/ Букина Татьяна Витальевна.data  
  inflating: embeddingsPickle/ Быкова Анна Андреевна.data  
  inflating: embeddingsPickle/ Василенко Юрий Владимирович.data  
  inflating: embeddingsPickle/ Викентьева Ольга Леонидовна.data  
  inflating: embeddingsPickle/ Володина Галина Емельяновна.data  
  inflating: embeddingsPickle/ Гагарина Динара Амировна.data  
  inflating: embeddingsPickle/ Гергерт Дмитрий Владимирович.data  
  inflating: embeddingsPickle/ Гущин Валерий Рафаилович.data  

In [3]:
import requests
import torch
from transformers import BertTokenizer, BertModel, BertConfig
from tqdm import tqdm
from summarizer import Summarizer

import sys

import pickle
import re
import math
import os

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#print(torch.cuda.get_device_name(0))

custom_config = BertConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)
#custom_model.to('cuda')
custom_model.eval()

modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [5]:
path = '/content/embeddingsPickle'
directories = os.listdir(path)
emb = {}
for file in directories:
    with open(path+'/'+file, "rb") as f:
        arr = pickle.load(f)
        emb[file.split('.')[0]] = arr

In [7]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open('/content/Курсовая работа Соломатин.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)
text = output_string.getvalue()
%time embeddings = modelSum.run_embeddings(text, num_sentences=10)

CPU times: user 26.7 s, sys: 803 ms, total: 27.5 s
Wall time: 27.3 s


In [ ]:
with open("TestDoc", 'r') as f:
    text = f.read()
    text = "".join([i.replace('\n',' ').replace('\t',' ') for i in text])
text
%time embeddings = modelSum.run_embeddings(text, num_sentences=10)

CPU times: user 55.7 s, sys: 1.23 s, total: 57 s
Wall time: 56.3 s


# Сумма всех

In [8]:
errorSum = {}
errorMae = {}
errorMse = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    sum = 0
    mae = 0
    mse = 0
    for i in range(len(embeddings)):
        for arrRow in range(len(arr)):
            for j in range(768):
                sum += embeddings[i][j] - arr[arrRow][j]
                mae += abs(embeddings[i][j] - arr[arrRow][j])
                mse += (embeddings[i][j] - arr[arrRow][j])**2
    errorSum[sum] = name
    errorMae[mae] = name
    errorMse[mse] = name
print()
print(errorSum[min(errorSum.keys())], errorMae[min(errorMae.keys())], errorMse[min(errorMse.keys())])

100%|██████████| 61/61 [00:39<00:00,  1.55it/s]


 Кычкин Алексей Владимирович  Артемьев Дмитрий Геннадьевич  Артемьев Дмитрий Геннадьевич


# Сумма минимальных предлжений с повторами

In [9]:
errorSum = {}
errorMae = {}
errorMse = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    allSum = 0
    allMae = 0
    allMse = 0
    for i in range(len(embeddings)):
        sum = [0]*len(arr)
        mae = [0]*len(arr)
        mse = [0]*len(arr)
        for arrRow in range(len(arr)):
            for j in range(768):
                sum[arrRow] += embeddings[i][j] - arr[arrRow][j]
                mae[arrRow] += abs(embeddings[i][j] - arr[arrRow][j])
                mse[arrRow] += (embeddings[i][j] - arr[arrRow][j])**2
        allSum += min(sum)
        allMae += min(mae)
        allMse += min(mse)
    errorSum[allSum] = name
    errorMae[allMae] = name
    errorMse[allMse] = name
print()
print(errorSum[min(errorSum.keys())], errorMae[min(errorMae.keys())], errorMse[min(errorMse.keys())])

100%|██████████| 61/61 [00:40<00:00,  1.52it/s]


 Божья-Воля Анастасия Александровна  Кычкин Алексей Владимирович  Кычкин Алексей Владимирович


# Сумма минимальных предложений без повторов

## Avg

In [10]:
errorSum = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    allSum = 0
    takenRow = []
    for i in range(len(embeddings)):
        if i in takenRow:
            continue
        sum = [0]*len(arr)
        for arrRow in range(len(arr)):
            for j in range(768):
                sum[arrRow] += embeddings[i][j] - arr[arrRow][j]
        allSum += min(sum)
        takenRow.append(sum.index(min(sum)))
    errorSum[allSum] = name
print()
print(errorSum[min(errorSum.keys())])

100%|██████████| 61/61 [00:06<00:00,  8.86it/s]


 Божья-Воля Анастасия Александровна


## Mae

In [11]:
errorMae = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    allMae = 0
    takenRow = []
    for i in range(len(embeddings)):
        if i in takenRow:
            continue
        mae = [0]*len(arr)
        for arrRow in range(len(arr)):
            for j in range(768):
                mae[arrRow] += embeddings[i][j] - arr[arrRow][j]
        allMae += min(mae)
        takenRow.append(mae.index(min(mae)))
    errorMae[allMae] = name
print()
print(errorMae[min(errorMae.keys())])

100%|██████████| 61/61 [00:06<00:00,  8.89it/s]


 Божья-Воля Анастасия Александровна


## Mse

In [12]:
errorMse = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    allMse = 0
    takenRow = []
    for i in range(len(embeddings)):
        if i in takenRow:
            continue
        mse = [0]*len(arr)
        for arrRow in range(len(arr)):
            for j in range(768):
                mse[arrRow] += embeddings[i][j] - arr[arrRow][j]
        allMse += min(mse)
        takenRow.append(mse.index(min(mse)))
    errorMse[allMse] = name
print()
print(errorMse[min(errorMse.keys())])

100%|██████████| 61/61 [00:06<00:00,  8.82it/s]


 Божья-Воля Анастасия Александровна


# Усредение эмбединнгов

In [13]:
newEmb = [0]*768
for arrEmb in embeddings:
    for j in range(len(arrEmb)):
        newEmb[j] += arrEmb[j] 
newEmb = [i/10 for i in newEmb]
errorSum = {}
errorMae = {}
errorMse = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    sum = 0
    mae = 0
    mse = 0
    newArr = [0]*768
    for arrEmb in arr:
        for j in range(len(arrEmb)):
            newArr[j] += arrEmb[j] 
    newArr = [i/10 for i in newArr]
    for j in range(768):
        sum += newEmb[j] - newArr[j]
        mae += abs(newEmb[j] - newArr[j])
        mse += (newEmb[j] - newArr[j])**2
    errorSum[sum] = name
    errorMae[mae] = name
    errorMse[mse] = name
print()
print(errorSum[min(errorSum.keys())], errorMae[min(errorMae.keys())], errorMse[min(errorMse.keys())])

100%|██████████| 61/61 [00:00<00:00, 129.35it/s]


 Кычкин Алексей Владимирович  Кычкин Алексей Владимирович  Гергерт Дмитрий Владимирович


# Минимальная разность

In [14]:
errorSum = {}
errorMae = {}
errorMse = {}
for name, arr in tqdm(emb.items(), position=0, leave=True):
    allSum = 0
    allMae = 0
    allMse = 0
    for i in range(len(embeddings)):
        sum = [0]*len(arr)
        mae = [0]*len(arr)
        mse = [0]*len(arr)
        for arrRow in range(len(arr)):
            for j in range(768):
                sum[arrRow] += embeddings[i][j] - arr[arrRow][j]
                mae[arrRow] += abs(embeddings[i][j] - arr[arrRow][j])
                mse[arrRow] += (embeddings[i][j] - arr[arrRow][j])**2
        allSum = min(sum)
        allMae = min(mae)
        allMse = min(mse)
    errorSum[allSum] = name
    errorMae[allMae] = name
    errorMse[allMse] = name
print()
print(errorSum[min(errorSum.keys())], errorMae[min(errorMae.keys())], errorMse[min(errorMse.keys())])

100%|██████████| 61/61 [00:40<00:00,  1.50it/s]


 Божья-Воля Анастасия Александровна  Андреева Ольга Юрьевна  Андреева Ольга Юрьевна
